In [4]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database
conn = create_engine(string).connect()

df = pd.read_sql(
    """
    select date, ticker, ret, roeq, mom12m
    from data
    where date>='2010-01' and date<='2017-12'
    order by date, ticker
    """, 
    conn
)
df = df.dropna()
df = df.set_index("date")

from pandas_datareader import DataReader as pdr

mkt = pdr(
  "F-F_Research_Data_Factors",
  "famafrench",
  start="2009-12",
  end="2017-12"
)

mkt = mkt[0] / 100
mkt.index = mkt.index.astype(str)
mkt["mkt"] = mkt["Mkt-RF"] + mkt["RF"]
mkt["lagmkt"] = mkt.mkt.shift()

df = df.join(mkt, how="left")

In [5]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import Ridge

model = TransformedTargetRegressor(
    transformer=QuantileTransformer(
      output_distribution="normal",
      random_state=0
    ),
    regressor=Ridge(random_state=0)
)

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
  QuantileTransformer(output_distribution="normal", random_state=0),
  PolynomialFeatures(degree=2),
  QuantileTransformer(output_distribution="normal", random_state=0),
  model
)

In [14]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from scipy.stats import spearmanr

spearman = make_scorer(
    lambda y1, y2 : spearmanr(y1, y2)[0]
)

param_grid = {
    "transformedtargetregressor__regressor__alpha": [0.01, 0.1, 1]
}

cv = GridSearchCV(
    pipe, 
    param_grid=param_grid,
    scoring=spearman,
    
)

X = df[["roeq", "mom12m", "lagmkt"]]
y = df.ret - df.mkt
_ = cv.fit(X, y)

In [15]:
cv.score(X, y)

0.07608450534420176

In [16]:
cv.best_params_

{'transformedtargetregressor__regressor__alpha': 0.1}

In [18]:
model = TransformedTargetRegressor(
    transformer=QuantileTransformer(
      output_distribution="normal",
      random_state=0
    ),
    regressor=Ridge(random_state=0, alpha=0.1)
)

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
  QuantileTransformer(output_distribution="normal", random_state=0),
  PolynomialFeatures(degree=2),
  QuantileTransformer(output_distribution="normal", random_state=0),
  model
)

_ = pipe.fit(X, y)


In [21]:
pipe.score(X, y)

-0.0009009483197663926